In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
import numpy as np

In [4]:
import csv

In [5]:
import pandas as pd

In [6]:
html_text = requests.get('https://www.dsebd.org/latest_share_price_scroll_l.php').text

In [7]:
soup = BeautifulSoup(html_text, 'lxml')

In [8]:
stocks_table = soup.find('div', class_ = 'table-responsive inner-scroll' )

In [9]:
stocks = stocks_table.find_all('tr')

In [10]:
stocks_data_list = []
loop_counter = 0
for stock in stocks:    
    
    stock_data = stock.find_all('td')
    
    stock_data_list = []
    nested_loop_counter = 0
    
    for data in stock_data:
            
        if 1 < nested_loop_counter < 8 :
            stock_data_list += [float(data.text.replace('--' , '0').replace(',',''))]
            
        elif nested_loop_counter == 1:
            stock_name = data.find('a')
            stock_data_list += [stock_name.text.strip()]
            
        elif nested_loop_counter == 8:
            stock_data_list += [int(data.text.replace(',','').replace('--', '0'))]
            
        elif nested_loop_counter == 9:
            stock_data_list += [float(data.text.replace(',','').replace('--','0'))]
                
        elif nested_loop_counter == 10:
            volume = int(data.text.replace(',',''))
            stock_data_list += [volume]
                
        nested_loop_counter += 1
        
    stocks_data_list.append(stock_data_list)

        
del stocks_data_list[0]

In [11]:
stocks_data_list[0][1:]

[6.5, 6.6, 6.4, 6.5, 6.5, 0.0, 111, 2.308, 355732]

In [12]:
market_update_dict = dict()
for stock in stocks_data_list:
    market_update_dict[stock[0]] = stock

In [13]:
len(market_update_dict)

384

In [14]:
listings = open('listings.csv', 'r')
market_update_for_wrangled = open('market_update_wrangled.csv', 'w')

with listings, market_update_for_wrangled:
    read_listings = csv.reader(listings)
    write_market_update = csv.writer(market_update_for_wrangled)
    
    for record in read_listings:
        try:
            write_market_update.writerow(market_update_dict[record[0]])
        except KeyError:
            continue

In [15]:
stocks_data_headings = stocks[0].find_all('th')

In [16]:
del stocks_data_headings[0]

stocks_headings_list = []

for heading in stocks_data_headings:
    stocks_headings_list += [heading.text]

In [17]:
stocks_headings_list

['TRADING CODE',
 'LTP*',
 'HIGH',
 'LOW',
 'CLOSEP*',
 'YCP*',
 'CHANGE',
 'TRADE',
 'VALUE (mn)',
 'VOLUME']

In [18]:
len(stocks_data_list)

384

In [19]:
df_mp_update = pd.read_csv('market_update_wrangled.csv', names = stocks_headings_list)

In [20]:
df_fin_disclosures = pd.read_csv('disclosures_wrangled.csv', names = ['TRADING CODE', 'SECTOR', 'CATEGORY', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS', 'FIN_YEAR'])

In [21]:
df_merged_mp_disclosures = pd.merge(df_mp_update, df_fin_disclosures, on = 'TRADING CODE')

In [22]:
df_merged_mp_disclosures

,TRADING CODE,LTP*,HIGH,LOW,CLOSEP*,YCP*,CHANGE,TRADE,VALUE (mn),VOLUME,SECTOR,CATEGORY,DIVIDEND,EPS_Q,EPS_Y,NAVPS,FIN_YEAR
0,1JANATAMF,6.5,6.6,6.4,6.5,6.5,0.0,111,2.308,355732,Mutual Funds,A,1.30,0.80,2.54,11.94,2021
1,1STPRIMFMF,19.7,20.1,19.7,19.8,19.9,-0.2,159,3.158,159618,Mutual Funds,A,0.80,1.48,0.38,11.62,2020
2,AAMRANET,44.8,46.3,44.7,44.8,45.4,-0.6,219,6.635,147103,IT Sector,A,2.77,2.04,2.14,37.09,2021
3,AAMRATECH,30.5,31.9,30.4,30.5,31.5,-1.0,442,17.719,572278,IT Sector,A,2.08,2.12,1.45,23.93,2021
4,ABB1STMF,5.6,5.8,5.6,5.6,5.7,-0.1,25,0.440,78250,Mutual Funds,A,0.80,0.68,2.61,11.97,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,VAMLBDMF1,7.9,7.9,7.8,7.9,7.8,0.1,67,2.021,257280,Mutual Funds,A,1.50,2.21,2.10,14.63,2021
275,VAMLRBBF,8.5,8.8,8.5,8.6,8.7,-0.2,25,0.293,33949,Mutual Funds,A,0.16,2.12,1.68,10.16,2020
276,VFSTDL,21.1,21.6,21.1,21.1,21.5,-0.4,212,6.659,313404,Textile,A,1.10,1.60,1.50,18.70,2021
277,WALTONHIL,1069.0,1080.0,1066.0,1069.0,1077.2,-8.2,1092,18.889,17631,Engineering,A,25.00,29.46,54.21,311.59,2021


In [23]:
df_for_lin_reg = df_merged_mp_disclosures.loc[:, ['TRADING CODE', 'LTP*', 'DIVIDEND', 'EPS_Q', 'EPS_Y', 'NAVPS','SECTOR']]

In [24]:
df_merged_mp_disclosures['DIVIDEND'].dtype

dtype('float64')

In [25]:
data_banks = df_for_lin_reg.groupby('SECTOR').get_group('Bank')

In [26]:
data_banks

,TRADING CODE,LTP*,DIVIDEND,EPS_Q,EPS_Y,NAVPS,SECTOR
5,ABBANK,13.4,0.68,0.59,0.50,32.26,Bank
18,ALARABANK,26.8,1.50,2.06,2.42,22.35,Bank
33,BANKASIA,20.4,1.00,3.15,1.74,23.34,Bank
51,BRACBANK,57.6,3.93,3.83,3.33,36.63,Bank
60,CITYBANK,27.2,3.13,3.96,4.29,29.03,Bank
74,DHAKABANK,14.3,1.47,2.18,2.20,21.69,Bank
80,DUTCHBANGL,77.3,13.12,8.33,10.00,58.65,Bank
83,EBL,38.5,8.50,5.61,5.15,36.28,Bank
94,EXIMBANK,12.9,1.07,1.57,1.99,21.66,Bank
98,FIRSTSBANK,12.5,1.14,1.73,2.93,18.91,Bank


In [27]:
X = np.column_stack((data_banks['DIVIDEND'], data_banks['NAVPS'], data_banks['EPS_Q'], data_banks['EPS_Y']))

In [28]:
y = data_banks['LTP*']

In [29]:
y

5      13.4
18     26.8
33     20.4
51     57.6
60     27.2
74     14.3
80     77.3
83     38.5
94     12.9
98     12.5
124    17.1
131    31.9
135    23.4
158    17.1
170    19.2
173     7.3
174    15.2
183    26.7
185    13.4
202    14.7
205    21.4
210    27.4
226    33.8
235    15.5
239    22.3
242    17.9
252    15.3
259    10.3
266    33.6
267    16.0
272    26.9
Name: LTP*, dtype: float64

In [30]:
from sklearn.linear_model import LinearRegression

In [31]:
from sklearn.metrics import r2_score

In [32]:
import statsmodels.api as sm

In [33]:
X2 = sm.add_constant(X)

In [34]:
est = sm.OLS(y, X2)

In [35]:
est2 = est.fit()

In [36]:
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                     25.60
Date:                Mon, 24 Jan 2022   Prob (F-statistic):           1.10e-08
Time:                        23:45:04   Log-Likelihood:                -101.01
No. Observations:                  31   AIC:                             212.0
Df Residuals:                      26   BIC:                             219.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3496      4.081      0.086      0.9

In [37]:
data_mf = df_for_lin_reg.groupby('SECTOR').get_group('Mutual Funds')

In [38]:
data_mf

,TRADING CODE,LTP*,DIVIDEND,EPS_Q,EPS_Y,NAVPS,SECTOR
0,1JANATAMF,6.5,1.30,0.80,2.54,11.94,Mutual Funds
1,1STPRIMFMF,19.7,0.80,1.48,0.38,11.62,Mutual Funds
4,ABB1STMF,5.6,0.80,0.68,2.61,11.97,Mutual Funds
16,AIBL1STIMF,8.8,1.23,0.21,1.83,11.57,Mutual Funds
31,ATCSLGF,9.8,1.50,1.12,1.93,12.41,Mutual Funds
57,CAPMBDBLMF,10.2,1.30,1.48,2.41,11.81,Mutual Funds
58,CAPMIBBLMF,16.0,1.35,1.28,1.89,11.97,Mutual Funds
70,DBH1STMF,7.5,1.20,0.13,2.13,11.79,Mutual Funds
84,EBL1STMF,7.5,1.30,2.13,2.58,11.76,Mutual Funds
85,EBLNRBMF,6.1,0.60,1.16,1.92,12.27,Mutual Funds


In [39]:
X_mf = np.column_stack((data_mf['DIVIDEND'], data_mf['NAVPS'], data_mf['EPS_Q'], data_mf['EPS_Y']))

In [40]:
y_mf = data_mf['LTP*']

In [41]:
X2_mf = sm.add_constant(X_mf)

In [42]:
est_mf = sm.OLS(y_mf, X2_mf)

In [43]:
est2_mf = est_mf.fit()

In [45]:
print(est2_mf.summary())

                            OLS Regression Results                            
Dep. Variable:                   LTP*   R-squared:                       0.414
Model:                            OLS   Adj. R-squared:                  0.338
Method:                 Least Squares   F-statistic:                     5.468
Date:                Mon, 24 Jan 2022   Prob (F-statistic):            0.00189
Time:                        23:45:44   Log-Likelihood:                -83.274
No. Observations:                  36   AIC:                             176.5
Df Residuals:                      31   BIC:                             184.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7604      2.691      0.283      0.7

In [50]:
df_for_lin_reg['SECTOR'].unique()

array(['Mutual Funds', 'IT Sector', 'Bank', 'Textile',
       'Pharmaceuticals & Chemicals', 'Insurance', 'Miscellaneous',
       'Food & Allied', 'Engineering', 'Fuel & Power',
       'Tannery Industries', 'Financial Institutions', 'Paper & Printing',
       'Telecommunication', 'Cement', 'Services & Real Estate',
       'Ceramics Sector', 'Travel & Leisure', 'Jute'], dtype=object)

In [55]:
data_ins = df_for_lin_reg.groupby('SECTOR').get_group('Insurance')

In [58]:
data_ins

,TRADING CODE,LTP*,DIVIDEND,EPS_Q,EPS_Y,NAVPS,SECTOR
15,AGRANINS,56.2,3.28,1.81,1.71,18.11,Insurance
29,ASIAINS,104.2,1.20,4.24,2.34,24.52,Insurance
30,ASIAPACINS,65.5,1.00,3.33,1.97,20.63,Insurance
47,BGIC,53.1,1.20,2.51,1.25,19.38,Insurance
48,BNICL,124.2,1.50,3.83,2.32,20.17,Insurance
59,CENTRALINS,52.4,4.50,2.68,2.18,25.02,Insurance
61,CITYGENINS,40.0,1.00,1.71,1.77,16.88,Insurance
64,CONTININS,45.0,2.39,2.27,1.29,19.84,Insurance
66,CRYSTALINS,51.6,1.00,2.46,4.59,20.98,Insurance
73,DGIC,38.2,1.00,1.32,0.91,7.16,Insurance


In [59]:
X_ins = np.column_stack((data_ins['DIVIDEND'], data_ins['NAVPS'], data_ins['EPS_Q'], data_ins['EPS_Y']))

In [63]:
y_ins = data_ins['LTP*']

15      56.2
29     104.2
30      65.5
47      53.1
48     124.2
59      52.4
61      40.0
64      45.0
66      51.6
73      38.2
75      70.0
81      36.9
88      32.9
96      34.2
104     52.7
110     97.1
133     62.3
137     44.9
139     40.6
159     48.0
179     58.9
181     51.7
189     73.3
192     57.1
195     55.7
198    109.4
201     89.2
206     79.6
208    118.4
211     38.0
221     84.7
223     49.3
227     42.6
251     69.8
258     71.7
261     60.8
270     62.0
Name: LTP*, dtype: float64